# QTM 531 – Midsemester Project
## Ao Lyu (Lorna)

In this project I will create and analyze a bage dataset in Amazon via web scraping. 

<font size = "5">

Import packages for data processing

In [63]:
# Manage datasets
import pandas as pd

# Work with time data
import time 

<font size = "5">

Import packages for HTML processing

In [64]:
# Conduct HTTP requests
import requests

# Construct tree structure of HTML data
import html5lib

# Parse HTML data obtained from scraping
from bs4 import BeautifulSoup as soup 
from bs4 import BeautifulSoup

<font size = "5">

Import packages for interactive <br>
website navigation

In [65]:
# Import webdriver for chrome
from webdriver_manager.chrome import ChromeDriverManager

# Automate navigating within browser (SELENIUM)
#------ Key: Manage keys
#------ Select: Obtain features from website
#------ WebDriverWait: Add wait times implicitly
#------ By: Use common information locator strategies
#------ EC and Options: Browser configuration
#------ remote.command: Check whether browser is active

from selenium import webdriver #to automate the navigating within the browser
from selenium.webdriver.common.keys    import Keys
from selenium.webdriver.support.ui     import Select
from selenium.webdriver.support.ui     import WebDriverWait 
from selenium.webdriver.common.by      import By
from selenium.webdriver.support        import expected_conditions as EC
from selenium.webdriver.chrome.options import Options #to use properties of the chrome webbrowser
from selenium.webdriver.remote.command import Command # Use to check whether the web driver is active


In [66]:
import os
import time
from datetime import date
import csv
from bs4 import BeautifulSoup
import subprocess
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By


# <span style="color:darkblue"> I. Running Chrome from Python </span>

In [95]:
base_url = 'https://www.etsy.com/c/jewelry'

In [14]:
# options = Options()

# # True hides the navigating of the browser by the scraper,
# # False shows you the tab/window opening and stuff getting clicked
# options.headless = False 

# driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

In [15]:
#driver.get(base_url)

In [16]:
# response = requests.get(base_url)
# html_content = response.text
# soup = BeautifulSoup(html_content, 'html.parser')

In [17]:
# driver = webdriver.Chrome(ChromeDriverManager().install())
# #driver.get("https://www.urbanoutfitters.com/dresses")
# driver.get("https://www.urbanoutfitters.com")

In [18]:
# from selenium.webdriver.common.by import By

# list_forms = driver.find_elements(By.CLASS_NAME, 'form-control')
# len(list_forms)

In [19]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# # Setup Selenium WebDriver
# options = webdriver.ChromeOptions()
# # Optional: run in headless mode
# # options.add_argument('--headless')
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# # Navigate to the website
# driver.get('https://www.urbanoutfitters.com/dresses')

# # Wait for the page to load (consider explicit waits for specific elements)
# driver.implicitly_wait(10)

# # Now you can extract the page source
# page_source = driver.page_source


# # Remember to close the driver after your scraping job is done
# driver.quit()


In [67]:
# Your HTML snippet
html_snippet = '''
<span aria-label="Original price: $79.00" class="c-pwa-product-price__current s-pwa-product-price__current">$79.00</span>
'''

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_snippet, 'html.parser')


price = soup.find('span', class_='c-pwa-product-price__current').text

print(price)  


$79.00


In [115]:
driver = webdriver.Chrome('path/to/chromedriver')

# Get the page where the dresses are listed
driver.get('https://www.etsy.com/c/jewelry')

# Once content is loaded, get the page source
html_content = driver.page_source

# Now you can use BeautifulSoup to parse the page source
soup = BeautifulSoup(html_content, 'html.parser')
# Find all elements that contain original prices
original_price_elements = soup.find_all('p', class_='wt-text-caption')

# # Extract the original prices
# original_prices = [elem.find('span', class_='wt-text-strikethrough').find('span', class_='currency-value').get_text() 
#                    for elem in original_price_elements 
#                    if elem.find('span', class_='wt-text-strikethrough') and elem.find('span', class_='currency-value')]

# # Find all elements that contain current prices
# current_price_elements = soup.find_all('p', class_='wt-text-title-01 lc-price')

# # Extract the current prices
# current_prices = [elem.find('span', class_='currency-value').get_text() for elem in current_price_elements if elem.find('span', class_='currency-value')]

# # Print out the results
# for original, current in zip(original_prices, current_prices):
#     print(f'Original Price: {original} | Current Price: {current}')

/var/folders/4d/z4q5k4qd0bgclyqnxbr4wkwr0000gn/T/ipykernel_95359/4144663846.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('path/to/chromedriver')


In [116]:
# Find all elements that could contain product prices
product_elements = soup.find_all('div', class_='v2-listing-card__info')  # Assuming this class wraps both original and current prices

original_prices = []
current_prices = []

for elem in product_elements:
    # Attempt to find the original price
    original_price_elem = elem.find('span', class_='wt-text-strikethrough')
    if original_price_elem and original_price_elem.find('span', class_='currency-value'):
        original_prices.append(original_price_elem.find('span', class_='currency-value').get_text())
    else:
        original_prices.append(pd.NA)  # Use pd.NA for missing values if using pandas, otherwise use None
    
    # Attempt to find the current price
    current_price_elem = elem.find('p', class_='wt-text-title-01 lc-price')
    if current_price_elem and current_price_elem.find('span', class_='currency-value'):
        current_prices.append(current_price_elem.find('span', class_='currency-value').get_text())
    else:
        current_prices.append(pd.NA)  # Similarly, handle missing current prices


In [124]:
import pandas as pd
from bs4 import BeautifulSoup

# Assuming soup has been created from your page source as shown previously

# Initialize lists to store product information
original_prices = []
current_prices = []
discounts = []

# Iterate over each product element to extract pricing and discount information
for elem in soup.find_all('div', class_='v2-listing-card__info'):
    # Attempt to find the original price
    original_price_elem = elem.find('span', class_='wt-text-strikethrough')
    if original_price_elem and original_price_elem.find('span', class_='currency-value'):
        original_price = original_price_elem.find('span', class_='currency-value').get_text()
        original_prices.append(original_price)
        
        # Since there's an original price, attempt to find the discount
        discount_span = elem.find('span', text=lambda text: text and '% off' in text)
        if discount_span:
            discount_text = discount_span.get_text(strip=True)
            discounts.append(discount_text)
        else:
            discounts.append(pd.NA)
    else:
        # If there is no original price, append NA to both original_prices and discounts
        original_prices.append(pd.NA)
        discounts.append(pd.NA)
    
    # Attempt to find the current price
    current_price_elem = elem.find('p', class_='wt-text-title-01 lc-price')
    if current_price_elem and current_price_elem.find('span', class_='currency-value'):
        current_price = current_price_elem.find('span', class_='currency-value').get_text()
        current_prices.append(current_price)
    else:
        current_prices.append(pd.NA)

# Assuming you have other data like product names, ratings, review counts, and URLs in corresponding lists
data = {
    'Name': product_names,  # This should be defined elsewhere in your code
    'Original Price': original_prices,
    'Current Price': current_prices,
    'Discount': discounts,
    # Add 'Rating', 'Review Count', 'URL', etc. as needed
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Now you can save the DataFrame to a CSV, display it, etc.
df.to_csv('/Users/aolyu/Documents/GitHub/qtm531spring2024/Assignment/midsemester_project/products_with_discounts.csv', index=False)

# Outputting the file path for convenience
print('/Users/aolyu/Documents/GitHub/qtm531spring2024/Assignment/midsemester_project/products_with_discounts.csv')


/Users/aolyu/Documents/GitHub/qtm531spring2024/Assignment/midsemester_project/products_with_discounts.csv


/var/folders/4d/z4q5k4qd0bgclyqnxbr4wkwr0000gn/T/ipykernel_95359/2138415147.py:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  discount_span = elem.find('span', text=lambda text: text and '% off' in text)


In [117]:
# Find all elements that contain product names
product_name_elements = soup.find_all('h2', class_='v2-listing-card__title')

# Extract the product names
product_names = [elem.get_text(strip=True) for elem in product_name_elements]

#for name in product_names:
    #print(name)


In [118]:
# Extract all data-star ratings
data_star_elements = soup.find_all('input', {'name': ['initial-rating', 'rating']})
data_star_ratings = [float(elem['value']) for elem in data_star_elements if elem.has_attr('value')]

# Extract all review counts
review_count_elements = soup.find_all('span', class_='wt-text-caption wt-text-gray wt-display-inline-block wt-nudge-l-3 wt-pr-xs-1')
review_counts = [elem.get_text(strip=True) for elem in review_count_elements]

# Display the results
for rating, count in zip(data_star_ratings, review_counts):
    print(f"Rating: {rating}, Review Count: {count}")

Rating: 4.7762, Review Count: (2,946)
Rating: 4.7762, Review Count: (6,821)
Rating: 4.8649, Review Count: (41,478)
Rating: 4.8649, Review Count: (79,471)
Rating: 4.8649, Review Count: (73,371)
Rating: 4.8649, Review Count: (4,131)
Rating: 4.905, Review Count: (3,402)
Rating: 4.905, Review Count: (12,724)
Rating: 4.8805, Review Count: (41,478)
Rating: 4.8805, Review Count: (73,371)
Rating: 4.845, Review Count: (15,530)
Rating: 4.845, Review Count: (14,609)
Rating: 4.9499, Review Count: (3,020)
Rating: 4.9499, Review Count: (14,609)
Rating: 4.8308, Review Count: (22,155)
Rating: 4.8308, Review Count: (19,981)
Rating: 4.8649, Review Count: (13,106)
Rating: 4.8649, Review Count: (107)
Rating: 4.8805, Review Count: (4,121)
Rating: 4.8805, Review Count: (7,040)
Rating: 4.9291, Review Count: (12,199)
Rating: 4.9291, Review Count: (913)
Rating: 4.8908, Review Count: (4,825)
Rating: 4.8908, Review Count: (15,083)
Rating: 4.9854, Review Count: (41,580)
Rating: 4.9854, Review Count: (12,724)
Rati

In [121]:
product_elements = soup.find_all('div', class_='v2-listing-card__info')
# Find all elements that may contain discount information
discount_elements = soup.find_all('p', class_='search-collage-promotion-price')

# Initialize an empty list for discounts
discounts = []

discount_elem = elem.find('p', class_='search-collage-promotion-price')
for elem in product_elements:
    if discount_elem and discount_elem.find('span', text=lambda text: text and '% off' in text):
        discount_text = discount_elem.find('span', text=lambda text: text and '% off' in text).get_text(strip=True)
        data['Discount'].append(discount_text)
    else:
        data['Discount'].append(pd.NA)

# # Loop through each element that could contain a discount
# for elem in discount_elements:
#     # Attempt to find the span containing the discount text
#     discount_span = elem.find('span', text=lambda text: text and '% off' in text)
#     if discount_span:
#         # Extract the text directly if found
#         discount_text = discount_span.get_text(strip=True)
#         discounts.append(discount_text)
#     else:
#         # Append NA if the discount is not present
#         discounts.append(pd.NA) 

In [123]:
# Assuming 'product_elements' represents each product in the listing
product_elements = soup.find_all('div', class_='v2-listing-card__info')  # Adjust the class as necessary

data = {
    'Name': [],
    'Original Price': [],
    'Current Price': [],
    'Discount': [],
    # Add more fields as needed
}

for elem in product_elements:
    # Extract product name
    name_elem = elem.find('h2', class_='v2-listing-card__title')
    data['Name'].append(name_elem.get_text(strip=True) if name_elem else pd.NA)
    
    # Extract original price
    original_price_elem = elem.find('span', class_='currency-value')
    data['Original Price'].append(original_price_elem.get_text(strip=True) if original_price_elem else pd.NA)
    
    # Extract current price
    current_price_elem = elem.find('p', class_='wt-text-title-01 lc-price')
    if current_price_elem and current_price_elem.find('span', class_='currency-value'):
        data['Current Price'].append(current_price_elem.find('span', class_='currency-value').get_text())
    else:
        data['Current Price'].append(pd.NA)
    
    # Extract discount
    discount_elem = elem.find('p', class_='search-collage-promotion-price')
    if discount_elem and discount_elem.find('span', text=lambda text: text and '% off' in text):
        discount_text = discount_elem.find('span', text=lambda text: text and '% off' in text).get_text(strip=True)
        data['Discount'].append(discount_text)
    else:
        data['Discount'].append(pd.NA)

# Create DataFrame
df = pd.DataFrame(data)

# Print first few rows to verify
print(df.head())

# Save to CSV
csv_path = '/Users/aolyu/Documents/GitHub/qtm531spring2024/Assignment/midsemester_project/products.csv'
df.to_csv(csv_path, index=False)

# Provide CSV path for download
csv_path


                                                Name Original Price  \
0  14k Gold Chain Necklace - Box Chain - Rope Cha...          28.80   
1  Turquoise Flower Stud Earrings, Sterling Silve...          40.02   
2  Large bi-color imperial topaz dangling earring...          39.50   
3  Custom Pet Portrait Necklace, Engraved Pet Por...          14.70   
4  Photo Locket Necklace, Folding Locket with Pic...          46.70   

  Current Price   Discount  
0         28.80  (52% off)  
1         40.02  (25% off)  
2         39.50       <NA>  
3         14.70  (70% off)  
4         46.70       <NA>  


/var/folders/4d/z4q5k4qd0bgclyqnxbr4wkwr0000gn/T/ipykernel_95359/2146275136.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  if discount_elem and discount_elem.find('span', text=lambda text: text and '% off' in text):
/var/folders/4d/z4q5k4qd0bgclyqnxbr4wkwr0000gn/T/ipykernel_95359/2146275136.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  discount_text = discount_elem.find('span', text=lambda text: text and '% off' in text).get_text(strip=True)


'/Users/aolyu/Documents/GitHub/qtm531spring2024/Assignment/midsemester_project/products.csv'

In [94]:
from bs4 import BeautifulSoup

# Assuming `soup` is a BeautifulSoup object containing the HTML of the page

# Find all elements that could indicate free shipping
shipping_badge_elements = soup.find_all('span', class_='wt-badge wt-badge--small wt-badge--statusValue')

# Extract free shipping information
free_shipping_info = []
for elem in shipping_badge_elements:
    if 'FREE shipping' in elem.get_text():
        # Free shipping is offered
        free_shipping_info.append("FREE shipping")
    else:
        # Free shipping is not offered
        free_shipping_info.append("No free shipping")

# Print the free shipping information to verify
for shipping in free_shipping_info:
    print(shipping)



FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping
FREE shipping


In [58]:
from bs4 import BeautifulSoup

# Assuming `soup` is a BeautifulSoup object containing the HTML of the page

# Placeholder for parent elements. You need to replace 'parent-element-class' with the actual class.
parent_elements = soup.find_all('div', class_='parent-element-class')

products_info = []

for parent in parent_elements:
    # Extracting product name
    name_elem = parent.find('h2', class_='v2-listing-card__title')
    product_name = name_elem.get_text(strip=True) if name_elem else "No Name"
    
    # Extracting star rating
    rating_elem = parent.find('input', {'name': 'rating'})
    star_rating = float(rating_elem['value']) if rating_elem else "No Rating"
    
    # Extracting review count
    review_count_elem = parent.find('span', class_='wt-text-caption wt-text-gray wt-display-inline-block wt-nudge-l-3 wt-pr-xs-1')
    review_count = review_count_elem.get_text(strip=True) if review_count_elem else "No Reviews"
    
    # Extracting discount
    discount_elem = parent.find('p', class_='wt-text-caption search-collage-promotion-price')
    if discount_elem:
        discount_text = discount_elem.find_all('span')[-1].get_text(strip=True)
        discount = discount_text if discount_text else "No Discount"
    else:
        discount = "No Discount"
    
    products_info.append({
        'Product Name': product_name,
        'Star Rating': star_rating,
        'Review Count': review_count,
        'Discount': discount
    })

# Displaying extracted information
for product in products_info:
    print(product)


In [22]:
dresses = soup.find_all('div', class_='dress-item')  # Example class name
for dress in dresses:
    price_span= soup.find('span', class_='c-pwa-product-price__current s-pwa-product-price__current')  
    price = dress.find('span', class_='price').text  # Example class name
    # Extract other details similarly
    print(f'Name: {name}, Price: {price}')


In [23]:
data = {'Name': [], 'Price': []}  # Add more fields as needed
for dress in dresses:
    data['Name'].append(dress.find('h2', class_='name').text)
    data['Price'].append(dress.find('span', class_='price').text)

df = pd.DataFrame(data)
print(df.describe())  # Basic statistics
df.to_csv('dresses.csv', index=False)  # Save to CSV


       Name  Price
count   0.0    0.0
mean    NaN    NaN
std     NaN    NaN
min     NaN    NaN
25%     NaN    NaN
50%     NaN    NaN
75%     NaN    NaN
max     NaN    NaN


In [24]:
from store_response import StoreResponseError, get_store_response
from scraper import scrape_store

def main():
    store_url = input("https://www.urbanoutfitters.com/womens-clothing")
    try:
        store_response = get_store_response(store_url)
        if store_response:
            scrape_store(store_response, store_url)
    except StoreResponseError as e:
        print(e)

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'store_response'

In [97]:
# Find all 'a' tags with class 'listing-link'
product_links = soup.find_all('a', class_='listing-link')

# Extract the href attributes to get the URLs
product_urls = [link['href'] for link in product_links]

# Print the URLs
for url in product_urls:
    print(url)

https://www.etsy.com/listing/462945859/raw-crystal-necklace-rose-quartz?click_key=79a1ad9dc4b9a58222eb036edc9ca734827ac211%3A462945859&click_sum=8f84eb60&ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=search_grid-1076529-1-1&pro=1&sts=1
https://www.etsy.com/listing/469875368/photo-locket-necklace-folding-locket?click_key=2a5e46c27227fe28e5d1ecdc8bfca8609d202390%3A469875368&click_sum=deecd1f6&ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=search_grid-1076529-1-2&frs=1&etp=1&sts=1
https://www.etsy.com/listing/546844015/custom-name-stethoscope-necklace-medical?click_key=7d95cb91d4404c68cb3557922e5cba94cff86787%3A546844015&click_sum=ac5d5ca2&ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=&ref=search_grid-1076529-1-3&pro=1
https://www.etsy.com/listing/159695340/initials-necklace-ampersand-or-heart?click_key=5b19739304e007063500cdc7373eef7254e5474d%3A159695340&click_sum=e137e360&ga_order

In [98]:
import pandas as pd

# Assuming you have these lists filled with your scraped data
data = {
    'Name': product_names,
    'Original Price': original_prices,
    'Current Price': current_prices,
    'Rating': data_star_ratings,
    'Review Count': review_counts,
    'Discount': discounts,
    'URL': product_urls
}

# Convert the data dictionary into a pandas DataFrame
df = pd.DataFrame(data)

# Display basic statistics of the dataset
print(df.describe())

# Save the dataset to a CSV file
df.to_csv('/Users/aolyu/Documents/GitHub/qtm531spring2024/Assignment/midsemester_project/products.csv', index=False)


ValueError: All arrays must be of the same length

In [99]:
# Find the maximum length among all lists
max_length = max(len(product_names), len(original_prices), len(current_prices), len(data_star_ratings), len(review_counts), len(discounts), len(product_urls))

# Function to pad lists
def pad_list(lst, length, pad_value=None):
    return lst + [pad_value] * (length - len(lst))

# Pad all lists
product_names = pad_list(product_names, max_length)
original_prices = pad_list(original_prices, max_length)
current_prices = pad_list(current_prices, max_length)
data_star_ratings = pad_list(data_star_ratings, max_length)
review_counts = pad_list(review_counts, max_length)
discounts = pad_list(discounts, max_length)
product_urls = pad_list(product_urls, max_length)

# Now you can create the DataFrame
data = {
    'Name': product_names,
    'Original Price': original_prices,
    'Current Price': current_prices,
    'Rating': data_star_ratings,
    'Review Count': review_counts,
    'Discount': discounts,
    'URL': product_urls
}
df = pd.DataFrame(data)
